In [1]:
# add path to explore_by_example src folder
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


# import usual libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import f1_score

# our system imports
from aideme.active_learning import *
from aideme.initial_sampling import *
from aideme.explore import *
from aideme.utils.metrics import three_set_metric, classification_metrics
from aideme.utils.plotting import plot_polytope
from aideme.utils.convergence import *

%matplotlib inline

In [2]:
def plot_fscore(metrics):
    df_list = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics]
    avg = sum([df['fscore'][~df['fscore'].isna()] for df in df_list]) / len(df_list)
    avg.plot(ylim=[0,1], marker='o')
    plt.show()

In [3]:
# DUMMY DATA
N = int(1e5)
dim = 2
limit = 2 * (0.001)**(1. / dim)
rng = np.random.RandomState(0)
X = rng.uniform(low=-2, high=2, size=(N, dim))  # do not forget to standardize the data. For this distribution, it should be fine without it.
y_subspace = np.vstack([np.abs(X[:, i]) < limit for i in range(dim)]).T.astype('float')  # partial labels (for each subspace)
y = y_subspace.min(axis=1)
user = DummyUser(y, y_subspace)

# visualize data distribution
print('selectivity :', 100 * y.sum() / len(y), '%')

#plt.figure(figsize=(10,8))
#plt.scatter(X[:, 0], X[:, 1], s=0.5, c=['b' if lb else 'r' for lb in y])
#plt.show()

selectivity : 0.105 %


# NO FACTORIZATION EXAMPLE

In [4]:
# SET-UP EXPLORATION CONFIGURATION
NUMBER_OF_ITERATIONS = 43  # number of points to be labeled by the user
SUBSAMPLING = float('inf')
INITIAL_SAMPLER = StratifiedSampler.from_user(user, pos=1, neg=1)  # start with one random positive sample and one random negative sample
#INITIAL_SAMPLER = RandomInitialSampler(10)
CALLBACK = [ # callback functions to be called at the end of each iteration
    classification_metrics(y, 'fscore'), 
    three_set_metric,
]
CALLBACK_SKIP = 10
PRINT_CALLBACK_RESULT = True
CONVERGENCE_CRITERIA = [
    max_iter_reached(NUMBER_OF_ITERATIONS),
    #all_points_are_known,
    #metric_reached_threshold('fscore', 0.8),
    metric_reached_threshold('tsm', 0.9),
]

explore = PoolBasedExploration(INITIAL_SAMPLER, SUBSAMPLING, CALLBACK, CALLBACK_SKIP, PRINT_CALLBACK_RESULT, CONVERGENCE_CRITERIA)

# CHOOSE AN ALGORITHM
#learner = RandomSampler(SVC(C=1e5, kernel='rbf', gamma='auto'))  # choose a random point
learner = SimpleMargin(C=1024, kernel='rbf')  # choose point closest to SVM decision boundary
#learner = KernelQueryByCommittee(kernel='rbf', sampling='deterministic', n_samples=8, warmup=1000, thin=100, rounding=True)  # version space algorithm
learner = DualSpaceModel(learner, sample_unknown_proba=0.5, mode='positive')  # Dual Space model


# RUN EXPLORATION
metrics = explore.run(X, user, learner, repeat=1)  # 'repeat' specifies how many times to repeat the exploration process

# COMPUTE AVERAGE F-SCORE OVER ALL REPEATS AND PLOT
#plot_fscore(metrics)
df = pd.DataFrame.from_dict({i: metric for i, metric in enumerate(metrics[0])}, orient='index')
df

iter: 1, fscore: 0.002595091569659672, tsm: 0.0
iter: 11, fscore: 0.028363047001620744, tsm: 0.0
iter: 21, fscore: 0.11247991430101767, tsm: 0.0
iter: 31, fscore: 0.05555555555555555, tsm: 3.313379426122683e-05
iter: 41, fscore: 0.5275590551181102, tsm: 0.00012597631645250694
iter: 44, fscore: 0.5507246376811594, tsm: 0.0003180350327820726


,phase,iter_time,labeled_indexes,final_labels,partial_labels,fit_time,get_next_time,fscore,tsm
0,begin,0.003805,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,initial_sampling,0.033990,"[28395, 19366]","[1.0, 0.0]","[[1.0, 1.0], [0.0, 0.0]]",0.001112,0.032878,0.002595,0.000000
2,exploration,0.035303,[33493],[0.0],"[[0.0, 0.0]]",0.000832,0.034471,NaN,NaN
3,exploration,0.034425,[13015],[0.0],"[[0.0, 0.0]]",0.000612,0.033813,NaN,NaN
4,exploration,0.036003,[53834],[0.0],"[[0.0, 0.0]]",0.000462,0.035542,NaN,NaN
5,exploration,0.038977,[27646],[0.0],"[[0.0, 0.0]]",0.000675,0.038301,NaN,NaN
6,exploration,0.036786,[168],[0.0],"[[0.0, 0.0]]",0.000480,0.036306,NaN,NaN
7,exploration,0.039362,[89673],[0.0],"[[0.0, 0.0]]",0.000609,0.038753,NaN,NaN
8,exploration,0.037687,[36268],[0.0],"[[0.0, 0.0]]",0.000759,0.036928,NaN,NaN
9,exploration,0.035125,[52148],[0.0],"[[0.0, 0.0]]",0.000499,0.034625,NaN,NaN


In [9]:
df.iloc[290:]

,phase,iter_time,labeled_indexes,final_labels,partial_labels,fit_time,get_next_time,fscore,tsm
290,initial_sampling,0.001929,"[7629, 82927, 91383, 62890, 1252, 94832, 78279...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",NaN,NaN,NaN,NaN
291,initial_sampling,0.040364,"[98511, 20959, 81462, 22722, 1036, 85846, 5196...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.005811,0.034553,0.000000,0.000000
292,exploration,0.078715,[42033],[1.0],"[[1.0, 1.0]]",0.041361,0.037355,NaN,NaN
293,exploration,0.063485,[70784],[1.0],"[[1.0, 1.0]]",0.023600,0.039884,NaN,NaN
294,exploration,0.068203,[28706],[1.0],"[[1.0, 1.0]]",0.022174,0.046029,NaN,NaN
295,exploration,0.072125,[34931],[1.0],"[[1.0, 1.0]]",0.029256,0.042869,NaN,NaN
296,exploration,0.061505,[18269],[1.0],"[[1.0, 1.0]]",0.017186,0.044318,NaN,NaN
297,exploration,0.150818,[38473],[1.0],"[[1.0, 1.0]]",0.029515,0.121302,NaN,NaN
298,exploration,0.086587,[61062],[1.0],"[[1.0, 1.0]]",0.026332,0.060255,NaN,NaN
299,exploration,0.151192,[75356],[0.0],"[[0.0, 1.0]]",0.097724,0.053469,NaN,NaN


# FACTORIZATION EXAMPLE

In [24]:
# SET-UP EXPLORATION CONFIGURATION
NUMBER_OF_ITERATIONS = 100  # number of points to be labeled by the user
SUBSAMPLING = float('inf')
INITIAL_SAMPLER = StratifiedSampler(pos=1, neg=1)  # start with one random positive sample and one random negative sample
CALLBACK = [ # callback functions to be called at the end of each iteration
    classification_metrics(y, 'fscore'), 
    three_set_metric,
]
CALLBACK_SKIP = 10
PRINT_CALLBACK_RESULT = True
CONVERGENCE_CRITERIA = [
    max_iter_reached(NUMBER_OF_ITERATIONS),
    #all_points_are_known,
    #metric_reached_threshold('fscore', 0.99),
    metric_reached_threshold('tsm', 0.9),
]

explore = PoolBasedExploration(INITIAL_SAMPLER, SUBSAMPLING, CALLBACK, CALLBACK_SKIP, PRINT_CALLBACK_RESULT, CONVERGENCE_CRITERIA)

# Factorized AL algorithms
PARTITION = [[i] for i in range(dim)]  #[[0], [1]]

# FACTORIZED VERSION SPACE
# label_function = 'AND', 'OR', 'PROD'
# loss = 'GREEDY', 'SQUARED', 'PRODUCT'
learner = SubspatialVersionSpace(warmup=100, thin=10, n_samples=8, rounding=True, kernel='rbf', gamma=None, partition=PARTITION, label_function='AND', loss='GREEDY')

# FACTORIZED SIMPLE MARGIN
#learner = SubspatialSimpleMargin(C=1024, kernel='rbf', gamma=5, partition=PARTITION, label_function='AND')

# FACTORIZED DSM
#learner = DualSpaceModel(SimpleMargin(C=1024, kernel='rbf'), mode='positive', sample_unknown_proba=0.5, partition=PARTITION)  # Dual Space model


# RUN EXPLORATION
metrics = explore.run(X, user, learner, repeat=1)  # use y_subspace here

# COMPUTE AVERAGE F-SCORE OVER ALL REPEATS AND PLOT
#plot_fscore(metrics)
df = pd.DataFrame.from_dict({i: metric for i, metric in enumerate(metrics[0])}, orient='index')
df

TypeError: __init__() missing 2 required positional arguments: 'pos_idx' and 'neg_idx'

In [12]:
0. != 0

False